In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

In [4]:
# 定义模型
pretrain_model_name = "bert-base-chinese"
model = AutoModelForSequenceClassification.from_pretrained(pretrain_model_name, num_labels=15)

Downloading: 100%|██████████| 624/624 [00:00<00:00, 325kB/s]
Downloading: 100%|██████████| 393M/393M [00:22<00:00, 18.6MB/s] 
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

In [5]:
# 定义分词器
tokenizer = AutoTokenizer.from_pretrained(pretrain_model_name)

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 14.8kB/s]
Downloading: 100%|██████████| 107k/107k [00:01<00:00, 79.9kB/s] 
Downloading: 100%|██████████| 263k/263k [00:04<00:00, 58.0kB/s] 


In [6]:
# 从本地加载数据
# 标签名要命名为 labels
raw_dataset = load_dataset(
    "csv",
    data_files={"train": "./data/train.csv", "dev": "./data/dev.csv"},
    delimiter="\t",
    column_names=["str_label", "text"],
)


Using custom data configuration default-ad69658844a650a2


100%|██████████| 2/2 [00:00<00:00, 1999.67it/s]


Dataset csv downloaded and prepared to C:\Users\zhenh\.cache\huggingface\datasets\csv\default-ad69658844a650a2\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 153.77it/s]


In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


# 分词
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)


100%|██████████| 10/10 [00:00<00:00, 14.46ba/s]


In [1]:
# 同样的, 将文本标签转换成整数
label_map = {
    "agriculture": 0,
    "house": 1,
    "game": 2,
    "sports": 3,
    "entertainment": 4,
    "story": 5,
    "car": 6,
    "culture": 7,
    "finance": 8,
    "edu": 9,
    "military": 10,
    "travel": 11,
    "stock": 12,
    "world": 13,
    "tech": 14,
}

In [9]:
def label_function(ex):
    ex["labels"] = label_map[ex["str_label"]]
    return ex


tokenized_datasets = tokenized_datasets.map(label_function, batched=False)

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
dev_dataset = tokenized_datasets["dev"].shuffle(seed=42)

100%|██████████| 10000/10000 [00:04<00:00, 2422.78ex/s]


In [18]:
# 加载评估器
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [23]:
# 定义训练参数
training_args = TrainingArguments("test_trainer_pt", evaluation_strategy="epoch", report_to="none", save_total_limit=2,)

# 定义训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
)


In [24]:
# 开始训练
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, str_label.
***** Running training *****
  Num examples = 53360
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 20010
  2%|▏         | 500/20010 [02:58<1:54:11,  2.85it/s]Saving model checkpoint to test_trainer_pt\checkpoint-500
Configuration saved in test_trainer_pt\checkpoint-500\config.json


{'loss': 1.6087, 'learning_rate': 4.8750624687656176e-05, 'epoch': 0.07}


Model weights saved in test_trainer_pt\checkpoint-500\pytorch_model.bin
  5%|▍         | 1000/20010 [05:53<1:48:41,  2.92it/s]Saving model checkpoint to test_trainer_pt\checkpoint-1000
Configuration saved in test_trainer_pt\checkpoint-1000\config.json


{'loss': 1.3419, 'learning_rate': 4.750124937531234e-05, 'epoch': 0.15}


Model weights saved in test_trainer_pt\checkpoint-1000\pytorch_model.bin
  7%|▋         | 1500/20010 [08:47<1:46:03,  2.91it/s]Saving model checkpoint to test_trainer_pt\checkpoint-1500
Configuration saved in test_trainer_pt\checkpoint-1500\config.json


...

Model weights saved in test_trainer_pt\checkpoint-19500\pytorch_model.bin
Deleting older checkpoint [test_trainer_pt\checkpoint-18500] due to args.save_total_limit
100%|█████████▉| 20000/20010 [9:35:29<00:01,  5.82it/s]Saving model checkpoint to test_trainer_pt\checkpoint-20000
Configuration saved in test_trainer_pt\checkpoint-20000\config.json


{'loss': 0.5392, 'learning_rate': 2.4987506246876563e-08, 'epoch': 3.0}


Model weights saved in test_trainer_pt\checkpoint-20000\pytorch_model.bin
Deleting older checkpoint [test_trainer_pt\checkpoint-19000] due to args.save_total_limit
100%|██████████| 20010/20010 [9:35:32<00:00,  5.46it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, str_label.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
                                                       
100%|██████████| 20010/20010 [9:36:38<00:00,  5.46it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 20010/20010 [9:36:38<00:00,  1.73s/it]

{'eval_loss': 1.1006031036376953, 'eval_accuracy': 0.66, 'eval_runtime': 66.0549, 'eval_samples_per_second': 151.389, 'eval_steps_per_second': 18.924, 'epoch': 3.0}
{'train_runtime': 34598.8993, 'train_samples_per_second': 4.627, 'train_steps_per_second': 0.578, 'train_loss': 0.8953476309359282, 'epoch': 3.0}


TrainOutput(global_step=20010, training_loss=0.8953476309359282, metrics={'train_runtime': 34598.8993, 'train_samples_per_second': 4.627, 'train_steps_per_second': 0.578, 'train_loss': 0.8953476309359282, 'epoch': 3.0})